In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from joblib import dump

In [2]:
dfpart = pd.read_csv("fish_participant.csv")
dfholdtest = pd.read_csv("fish_holdout_demo.csv")
df= pd.concat([dfpart, dfholdtest])

In [3]:
df.Species.unique()

array(['Bream', 'Perch', 'Roach', 'Parkki', 'Smelt', 'Whitefish', 'Pike'],
      dtype=object)

In [4]:
#All fish are slender fish. Calculation below derived from information 
#at https://www.almanac.com/content/how-estimate-weight-fish for slender fish. I am using, to predict the girth,
#an approxomation forumla I found here at https://www.fishswami.com/fish_weight_calculator, which
#offers up approxgirth= .58*length. I am using length3 as my length in my model per the Farmer's Almanac link, which
#specifies a tip to tip length should be used (presumably the longest number, which is length3 in all instances.
#I can presume weights are in grams and measurements are in cm given that the measures don't line up for the small fish
#these are (as I discovered on Googling).


In [5]:
df = df.assign(length_in_inches = (df["Length3"]*0.39370))
df = df.assign(weightfactor = (df["length_in_inches"]*(df["length_in_inches"]*.58)*(df["length_in_inches"]*.58)/900))

In [6]:
df

,Species,Weight,Length1,Length2,Length3,Height,Width,length_in_inches,weightfactor
0,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340,13.38580,0.896492
1,Perch,110.0,20.0,22.0,23.5,5.5225,3.9950,9.25195,0.296015
2,Roach,160.0,20.5,22.5,25.3,7.0334,3.8203,9.96061,0.369378
3,Parkki,60.0,14.3,15.5,17.4,6.5772,2.3142,6.85038,0.120159
4,Bream,700.0,30.4,33.0,38.3,14.8604,5.2854,15.07871,1.281463
...,...,...,...,...,...,...,...,...,...
23,Perch,1000.0,39.8,43.0,45.2,11.9328,7.2772,17.79524,2.106323
24,Bream,720.0,32.0,35.0,40.6,16.3618,6.0900,15.98422,1.526468
25,Perch,1100.0,39.0,42.0,44.6,12.8002,6.8684,17.55902,2.023552
26,Perch,100.0,16.2,18.0,19.2,5.2224,3.3216,7.55904,0.161441


In [7]:
X = df[["length_in_inches", "weightfactor"]].values
y = df[["Weight"]].values.ravel()

In [8]:
#df.to_csv("graphingdoc.csv")
#this let me export to a CSV and later go into Excel and see the best fit regression using a graphed trendline and why.
#Linear regression is not good here. I noticde that my model worked well at 
#lower weights but seems not to work so well at higher weights.

#Scrolling through SciKitLearn documentation, I found a decision tree regressor, which by far gave me the best MSE and r2
#score when I tested the participant data against the test demo data.

#Credit to https://stackoverflow.com/questions/34165731/a-column-vector-y-was-passed-when-a-1d-array-was-expected for
#helping me clear the associated warning

In [9]:
rg = DecisionTreeRegressor()
rg.fit(X, y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [10]:
dump(rg, "reg.joblib")

['reg.joblib']